**Notes**
1. To reproduce images provided in our submission, the ideal runtime enviornment for this Jupyter notebook is Google Colab using an A100 GPU.
2. You may need a Hugging Face account to download the LoRA adapters, but registeration is free.
3. If you run into an issue with insufficient GPU memory (which might happen if you use an L4 GPU instead of A100), try decreasing the size of a batch to 1 or 2.
4. Make sure to configure the working directory of your jupyter notebook as the project root directory (NOT the `notebooks` folder) to avoid importing errors

In [1]:
# Install and load necessary packages
# %%capture
# !pip install accelerate diffusers transformers
# !pip install -U peft
# !pip install git+https://github.com/openai/CLIP.git

import torch
from diffusers import StableDiffusionXLPipeline
import matplotlib.pyplot as plt
import random

In [ ]:
# get the available cuda devices
print(torch.cuda.is_available())

In [ ]:
%%capture
PROFESSION = 'mathematics scientist'
positive_prompt = f"a photo of a {PROFESSION}, looking at the camera, ultra quality, sharp focus"
negative_prompt = "cartoon, anime, 3d, painting, b&w, low quality"
adapters = ["white_male", "asian_female"]

model_dict = {
    "asian_female": "NYUAD-ComNets/Asian_Female_Profession_Model",
    "white_male": "NYUAD-ComNets/White_Male_Profession_Model"
}
models = [model_dict[name] for name in adapters]
pipeline = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", variant="fp16", use_safetensors=True, torch_dtype=torch.float16).to("cuda")
for i,j in zip(models,adapters):
    pipeline.load_lora_weights(i, weight_name="pytorch_lora_weights.safetensors",adapter_name=j)

In [ ]:
pipeline.set_adapters(adapters, adapter_weights=[0.5, 0.5]) # merge the two LoRA checkpoints

TOTAL_SIZE, NUM_BATCHES = 32, 4

generator = torch.Generator(device="cuda")
generator.manual_seed(0)
torch.cuda.empty_cache()

all_images = []
for i in range(NUM_BATCHES):
    generator.manual_seed(i)
    all_images += pipeline(
        prompt=positive_prompt,
        negative_prompt=negative_prompt,
        num_inference_steps = 100,
        num_images_per_prompt = int(TOTAL_SIZE / NUM_BATCHES),
        generator = generator,
        height = 1024,
        width = 1024
    ).images
    torch.cuda.empty_cache()

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(16, 16))
for ax, img in zip(axs.flatten(), all_images[:16]):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()